In [1]:
using Statistics
X = rand(Float32, 10^7)

x = copy(X); y = quantile!(x, [0.02, 0.98])
x = copy(X); z = [quantile!(x, 0.02), quantile!(x, 0.98)]
@show y == z;

y == z = true


In [2]:
x = copy(X); @time quantile!(x, [0.02, 0.98])
x = copy(X); @time quantile!(x, [0.02, 0.98])
x = copy(X); @time quantile!(x, [0.02, 0.98]);

  1.162702 seconds (2 allocations: 192 bytes)
  1.168330 seconds (2 allocations: 192 bytes)
  1.161032 seconds (2 allocations: 192 bytes)


In [3]:
x = copy(X); @time [quantile!(x, 0.02), quantile!(x, 0.98)]
x = copy(X); @time [quantile!(x, 0.02), quantile!(x, 0.98)]
x = copy(X); @time [quantile!(x, 0.02), quantile!(x, 0.98)];

  0.163733 seconds (3 allocations: 128 bytes)
  0.190804 seconds (3 allocations: 128 bytes)
  0.179826 seconds (3 allocations: 128 bytes)


In [4]:
# `@time`'s added version of https://github.com/JuliaLang/Statistics.jl/blob/master/src/Statistics.jl#L949-L959

function myquantile!(v::AbstractVector, p::Union{AbstractArray, Tuple{Vararg{Real}}};
                   sorted::Bool=false, alpha::Real=1., beta::Real=alpha)
    if !isempty(p)
        minp, maxp = extrema(p)
        print("Statistics._quantilesort!(v, sorted, minp, maxp):             ")
        @time Statistics._quantilesort!(v, sorted, minp, maxp)
    end
    print("map(x->Statistics._quantile(v, x, alpha=alpha, beta=beta), p):")
    return @time map(x->Statistics._quantile(v, x, alpha=alpha, beta=beta), p)
end

myquantile!(v::AbstractVector, p::Real; sorted::Bool=false, alpha::Real=1., beta::Real=alpha) = begin
    print("Statistics._quantilesort!(v, sorted, p, p):        ")
    @time Statistics._quantilesort!(v, sorted, p, p)
    print("Statistics._quantile(v, p, alpha=alpha, beta=beta):")
    @time Statistics._quantile(v, p, alpha=alpha, beta=beta)
end

println("myquantile!(x, [0.02, 0.98]):")
x = copy(X); u = myquantile!(x, [0.02, 0.98])
println("\n[myquantile!(x, 0.02), myquantile!(x, 0.98)]:")
x = copy(X); v = [myquantile!(x, 0.02), myquantile!(x, 0.98)]
println(); @show u == v;

myquantile!(x, [0.02, 0.98]):
Statistics._quantilesort!(v, sorted, minp, maxp):               1.165920 seconds
map(x->Statistics._quantile(v, x, alpha=alpha, beta=beta), p):  0.000001 seconds (1 allocation: 96 bytes)

[myquantile!(x, 0.02), myquantile!(x, 0.98)]:
Statistics._quantilesort!(v, sorted, p, p):          0.079420 seconds
Statistics._quantile(v, p, alpha=alpha, beta=beta):  0.000000 seconds
Statistics._quantilesort!(v, sorted, p, p):          0.085581 seconds
Statistics._quantile(v, p, alpha=alpha, beta=beta):  0.000000 seconds

u == v = true


In [5]:
x = copy(X); @time y = quantile!(x, [0.499, 0.501])
x = copy(X); @time y = quantile!(x, [0.499, 0.501])
x = copy(X); @time y = quantile!(x, [0.499, 0.501]);

  0.133984 seconds (2 allocations: 192 bytes)
  0.138859 seconds (2 allocations: 192 bytes)
  0.161751 seconds (2 allocations: 192 bytes)


In [6]:
x = copy(X); @time z = [quantile!(x, 0.499), quantile!(x, 0.501)]
x = copy(X); @time z = [quantile!(x, 0.499), quantile!(x, 0.501)]
x = copy(X); @time z = [quantile!(x, 0.499), quantile!(x, 0.501)];

  0.235203 seconds (3 allocations: 128 bytes)
  0.225585 seconds (3 allocations: 128 bytes)
  0.200007 seconds (3 allocations: 128 bytes)


In [7]:
@which quantile!(x, [0.02, 0.98])

quantile!(v::AbstractVector{T} where T, p::Union{Tuple{Vararg{Real, N} where N}, AbstractArray}; sorted, alpha, beta) in Statistics at D:\Julia-1.6.2\share\julia\stdlib\v1.6\Statistics\src\Statistics.jl:934

In [8]:
@which quantile!(x, 0.02)

quantile!(v::AbstractVector{T} where T, p::Real; sorted, alpha, beta) in Statistics at D:\Julia-1.6.2\share\julia\stdlib\v1.6\Statistics\src\Statistics.jl:943

In [9]:
@which sort!(x, 1, length(x), Base.Sort.PartialQuickSort(2*10^5:98*10^5), Base.Sort.Forward)

sort!(v::AbstractVector{T} where T, lo::Integer, hi::Integer, a::PartialQuickSort, o::Base.Order.Ordering) in Base.Sort at sort.jl:623